#### Import rebalanced dataset

In [12]:
import pandas as pd
import os
import numpy as np

os.chdir('/Users/owenxoual/Desktop/DataHEC/quintenProject/unbalanced_data/dataset/raw')
!pwd

/Users/owenxoual/Desktop/DataHEC/quintenProject/unbalanced_data/dataset/raw


In [13]:
df = pd.read_csv("creditcard.csv")

#### Load functions from function.py script

In [14]:
os.chdir('/Users/owenxoual/Desktop/DataHEC/quintenProject/unbalanced_data/src')
!pwd

/Users/owenxoual/Desktop/DataHEC/quintenProject/unbalanced_data/src


In [15]:
import function
import master

#### Resample datasets

In [19]:
from function import smote_enn, smote_tomek

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = getdataset(df)

# Resample the training dataset
X_res, y_res = smote_enn(X_train, y_test, strategy = "auto")
#X_res1, y_res1 = smote_tomek(X_train, y_test, strategy = "auto")

ValueError: Found input variables with inconsistent numbers of samples: [199364, 85443]

#### Random Forest - Randomized Grid Search for hyperparameter tuning

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid_RF = grid_search_dict_RF(n_estimators, 
                                     max_features,
                                     max_depth,
                                     min_samples_split,
                                     min_samples_leaf,
                                     bootstrap)

In [ ]:
best_param_RF = grid_search_random(X_res,
                                   y_res,
                                   method = "RF",
                                   random_grid_RF,
                                   n_iter = 100,
                                   cv = 3)

print(best_param_RF)